<a href="https://colab.research.google.com/github/Tower5954/TensorFlow/blob/main/06_Transfer_Learning_Scaling_Up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning part.3

What we are going to cover:
* Downloading and preparing 10% of all Food101 classes (7500+ training images)
* Training a transfer learning feature extraction model
* Fine-tuning a feature extraction model to beat the original Food101 paper with only 10% of the data.
* Evaluating "Food Visions mini" predictions.
  * Finding the most wrong predictions (on the test data)
* Making predictions with Food Vision mini on our own custom data.

We have now seen the power of transfer learning feature extraction and fine-tuning, now it is time to scale up to all of the classes in food101 (101 total classes of food).

The goal is to beat the original food 101 paper with only 10% of the training (leveraging the power of deep learning).

 [Original Food101 paper.](https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/)

 The baseline to beat is 50.76% accuracy across 101 classes.

In [1]:
# Set up GPU 
# Check to see if running a GPU

!nvidia-smi

Wed Jan 19 07:00:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------